# Baza BASiW
Przypadki i zgony w podziale na:
* Datę
* Województwo
* Powiat
* Wiek
* Płeć
* Stan zaszczepienia
* Choroby współistniejące
* Upośledzenie odporności
* Więcej...

## ŚCIĄGNIJ AKTUALNE DANE RĘCZNIE!!!
https://basiw.mz.gov.pl/index.html#/visualization?id=3761

In [1]:
"""
Set PYTHONPATH for modules in parallel directory

https://stackoverflow.com/questions/3108285/in-python-script-how-do-i-set-pythonpath/3108301

"""

import sys

try:
    sys.path.index('..') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('..') # Or os.getcwd() for this directory

#--------------------------------------------------------------
import pandas as pd
from share.helper_functions import was_modified_today, display_all
import plotly.express as px
import plotly.graph_objects as go
import os
import datetime


In [2]:
class RawDf:
    df : pd.DataFrame
    def __init__(self, df : pd.DataFrame) -> None:
        self.df=df

class BASiWformat:
    image_dir : str
    data_dir : str
    data_file_deaths : str
    data_file_cases : str
    teryt_file : str
    path_deaths : str
    path_cases : str
    path_teryt : str
    _dfd_raw : RawDf
    _dfc_raw : RawDf
    _df_teryt_raw : RawDf
    df_woj : pd.DataFrame
    teryt_dict : dict
    dfd : pd.DataFrame

    def __init__(self,
        image_dir : str,
        data_dir : str,
        data_file_deaths : str,
        data_file_cases : str,
        teryt_file : str
        ) -> None:

        self.image_dir=image_dir
        self.data_dir=data_dir
        self.data_file_deaths=data_file_deaths
        self.data_file_cases=data_file_cases
        self.teryt_file=teryt_file
        self.path_deaths = os.sep.join([self.data_dir,self.data_file_deaths])
        self.path_cases = os.sep.join([self.data_dir,self.data_file_cases])
        self.path_teryt = os.sep.join([self.data_dir,self.teryt_file])
        self.read_data()
        self.make_teryt_dict()
        self.dfd = self.format_df('data_rap_zgonu', self._dfd_raw)
        return


    def read_data(self) -> None:
        # if was_modified_today(path_deaths):
        #     print('Data up to date')
        #     dfd = pd.read_csv(path_deaths, encoding = 'cp1250', sep = ';')
        #     dfc = pd.read_csv(path_cases, encoding = 'cp1250', sep = ';')
        # else:
        #     print('Old data, need to download new data!')
        self._dfd_raw = RawDf(pd.read_csv(self.path_deaths, encoding = 'cp1250', sep = ';',low_memory=False))
        self._dfc_raw= RawDf(pd.read_csv(self.path_cases, encoding = 'cp1250', sep = ';', low_memory=False))
        self._df_teryt_raw= RawDf(pd.read_csv(self.path_teryt, sep = ';'))
        return
        
    def make_teryt_dict(self) -> None:
        self.df_woj = self._df_teryt_raw.df.query('NAZWA_DOD == "województwo"')[['WOJ', 'NAZWA']]
        self.df_woj['NAZWA'] = self.df_woj['NAZWA'].str.title()
        self.df_woj.index = self.df_woj.pop('WOJ')
        self.teryt_dict = self.df_woj.to_dict()['NAZWA']
        return

    def format_df(self, col : str, df_raw : RawDf) -> pd.DataFrame:
        df = df_raw.df.copy(deep=True)
        df[col] = pd.to_datetime(df[col], format = "%Y-%m-%d")
        df['Województwo'] = df['teryt_woj']
        df['Województwo'].replace(self.teryt_dict, inplace=True)
        return df

    def dfd_from(self, start_date : datetime.date) -> pd.DataFrame:
        return self.dfd[self.dfd['data_rap_zgonu'] >= pd.to_datetime(start_date)]
        

In [3]:
basiw = BASiWformat(
    image_dir = './images',
    data_dir = './basiw_data',
    data_file_deaths = 'zgony.csv',
    data_file_cases = 'zakazenia.csv',
    teryt_file = 'TERC_Urzedowy_2021-12-14.csv'
    )

basiw.dfd


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [4]:
wave4_date = datetime.date(2021, 7, 14)
dfd4 = basiw.dfd_from(wave4_date)
dfd4


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
45248,2021-07-14,2,201,K,95.0,95+,N,NaN,NaN,N,1,N,Dolnośląskie
45249,2021-07-14,4,410,K,89.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
45250,2021-07-14,14,1412,K,45.0,45-54,N,NaN,NaN,N,1,N,Mazowieckie
45251,2021-07-14,16,1610,M,65.0,65-74,T,Astra Zeneca,jedna_dawka,N,1,N,Opolskie
45252,2021-07-14,18,1808,M,74.0,65-74,T,NaN,NaN,N,1,N,Podkarpackie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


### Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

In [5]:
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')
dfpopwoj['Województwo'] = dfpopwoj['Województwo'].str.title()
dfpopwoj.drop(['TERYT'], axis='columns', inplace=True)
dfpopwoj.index = dfpopwoj['Województwo']
dfpopwoj.drop(['Województwo'], axis='columns' , inplace = True)
pop_dict = dfpopwoj.to_dict()['Ludność']
# pop_dict

In [6]:
 def prepare_df(deaths_df: pd.DataFrame, from_date : datetime.date) -> pd.DataFrame:
    base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

    df = base_df.groupby('Województwo').sum()
    df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Województwo'] = df.index
    df['Ludność'] = df['Województwo'].replace(pop_dict)
    # df

    Q = f'(w_pelni_zaszczepiony == "N")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony nie w pełni zaszczepionych'] = df1

    Q = f'(w_pelni_zaszczepiony == "T")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony w pełni zaszczepionych'] = df1




    df['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony nie w pełni zaszczepionych'] / df['Ludność'] * 1e5
    df['Zgony w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony w pełni zaszczepionych'] / df['Ludność'] * 1e5

    df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
    return df

    
def plot_df(from_date : datetime.date, df : pd.DataFrame, image_dir : str) -> pd.DataFrame:
    nie = 'Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'
    tak = 'Zgony w pełni zaszczepionych na 100 tys. mieszkańców'

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df['Województwo'], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df['Województwo'], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br>\
        w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_woj_od'+str(from_date)+'.jpg']), scale=4)
    return


In [7]:
from_date = datetime.date(2021,7,14)

df = prepare_df(basiw.dfd, from_date)
plot_df(from_date, df, basiw.image_dir)

### Liczba zgonów na COVID-19 w Polsce na 100 tys. mieszkańców<br> w podziale na grupy wiekowe<br> (Od 14.07.2021)

Potem dla województw

**Potrzebny jest słownik z danymi: Populacja Polski / regionu wg grup wiekowych !!!**

In [28]:
def group_by(base_df: pd.DataFrame, col: str) -> pd.DataFrame:
   df = base_df.groupby(col).sum()
   df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   # df[col] = df.index
   # df['Ludność'] = df[col].replace(pop_dict)   ##################### !!!
   # df

   Q = f'(w_pelni_zaszczepiony == "N")'

   df1 = base_df.query(Q).groupby(col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony nie w pełni zaszczepionych'] = df1

   Q = f'(w_pelni_zaszczepiony == "T")'

   df1 = base_df.query(Q).groupby(col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony w pełni zaszczepionych'] = df1
   return df
 
 
def prepare_df2(deaths_df: pd.DataFrame, col: str, from_date : datetime.date) -> pd.DataFrame:
   base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

   df = group_by(base_df, col)
   display(df)

   df['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony nie w pełni zaszczepionych'].to_numpy() / df['Ludność'].to_numpy() * 1e5
   # display(df)
   
   # df['Zgony w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony w pełni zaszczepionych'].to_numpy() / df['Ludność'] * 1e5

   # df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
   return df

In [27]:
from_date = datetime.date(2021,7,14)

df = prepare_df2(basiw.dfd, 'kat_wiek', from_date)

,liczba_zaraportowanych_zgonow,kat_wiek,Ludność,Zgony nie w pełni zaszczepionych,Zgony w pełni zaszczepionych
kat_wiek,,,,,
0-18,9,0-18,0-18,8,1.0
19-24,18,19-24,19-24,17,1.0
25-34,99,25-34,25-34,92,7.0
35-44,253,35-44,35-44,228,25.0
45-54,622,45-54,45-54,542,80.0
55-64,1465,55-64,55-64,1172,293.0
65-74,3565,65-74,65-74,2432,1133.0
75-84,3929,75-84,75-84,2714,1215.0
85-94,3536,85-94,85-94,2622,914.0


TypeError: unsupported operand type(s) for /: 'int' and 'str'